In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                        google-cloud-storage \
                                        "numpy<2" \
                                        google-cloud-pipeline-components

In [2]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

LOCATION = "us-central1"
LOCATION = "us-central1"  # @param {type:"string"}

In [3]:
BUCKET_URI = f"gs://{PROJECT_ID}-unique"  # @param {type:"string"}
PIPELINE_ROOT = f"{BUCKET_URI}/pca_pipeline"

In [4]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://my-project-0004-346516-unique/...
ServiceException: 409 A Cloud Storage bucket named 'my-project-0004-346516-unique' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [5]:
PIPELINE_ROOT = f"{BUCKET_URI}/pca_pipeline"  # This is where all pipeline artifacts are sent. You'll need to ensure the bucket is created ahead of time
PIPELINE_ROOT
print(f"PIPELINE_ROOT: {PIPELINE_ROOT}")

PIPELINE_ROOT: gs://my-project-0004-346516-unique/pca_pipeline


In [6]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [7]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 255766800726-compute@developer.gserviceaccount.com


In [8]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://my-project-0004-346516-unique/
No changes made to gs://my-project-0004-346516-unique/


In [9]:
from typing import NamedTuple

import kfp
from google.cloud import aiplatform
from kfp import compiler, dsl
from kfp.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                     OutputPath, component)

In [10]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [11]:
import kfp
from kfp.dsl import component
from kfp.dsl import Output, Metrics
from typing import NamedTuple
import os

@component(
    packages_to_install=[
        "scikit-learn==1.0.2",
        "pandas==1.3.5",
        "matplotlib==3.5.1",
        "numpy<2"
    ],
    base_image="python:3.9")
# def perform_pca(  Output[Dataset],metrics: Output[Metrics], n_components: int = 2):
def perform_pca( data: Output[Dataset],metrics: Output[Metrics], n_components: int = 2):
    
    """
    Performs PCA on the Iris dataset, saves the transformed data,
    and generates a scree plot.

    Args:
        n_components (int): The number of principal components to retain.

    Returns:
        None
    """
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.decomposition import PCA
    from sklearn.datasets import load_iris
    from sklearn.preprocessing import StandardScaler
    import os

    # Load the Iris dataset
    iris = load_iris()
    X = iris.data
    y = iris.target

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Perform PCA
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X_scaled)

    # Create a Pandas DataFrame for the transformed data
    df = pd.DataFrame(X_reduced, columns=[f'PC{i+1}' for i in range(n_components)])
    df['target'] = y  # Add the target variable for potential analysis

    # Ensure the output directory exists
    os.makedirs(data.path, exist_ok=True)

    # Save the transformed data to a CSV file
    output_path = os.path.join(data.path, "pca_transformed_data.csv")
    df.to_csv(output_path, index=False)

    # Explained variance ratio
    explained_variance = pca.explained_variance_ratio_
    print("Explained Variance Ratio:", explained_variance)

    # Log metrics
    metrics.log_metric("explained_variance_ratio_PC1", explained_variance[0])
    if n_components > 1:
        metrics.log_metric("explained_variance_ratio_PC2", explained_variance[1])

    # Cumulative explained variance
    cum_var_exp = np.cumsum(explained_variance)

    # Scree plot
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(explained_variance) + 1), explained_variance, 'ro-', linewidth=2)
    plt.title('Scree Plot')
    plt.xlabel('Principal Component')
    plt.ylabel('Variance Explained')
    plt.savefig(os.path.join(data.path, "scree_plot.png"))  # Save the plot to a file
    plt.close()

    return


SyntaxError: invalid syntax (1755134344.py, line 15)

In [ ]:
@dsl.pipeline(
    name="pca-pipeline",
    description="A pipeline that performs PCA on the Iris dataset"
)
def pca_pipeline():
    pca_task = perform_pca(
        n_components=2
    )
    pca_task.set_caching_options(enable_caching=False)

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pca_pipeline, package_path="pca_pipeline.yaml"
)

In [ ]:
job = aiplatform.PipelineJob(
    display_name="pca-pipeline",
    template_path="pca_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
)

job.run()